<a href="https://colab.research.google.com/github/alec-carawan/Intro-to-Machine-Learning/blob/main/HW5_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:

%matplotlib inline
import numpy as np
import torch
torch.set_printoptions(edgeitems=2, linewidth=75)

In [45]:
t_c = torch.tensor([0.5, 14.0, 15.0, 28.0, 11.0,
                    8.0, 3.0, -4.0, 6.0, 13.0, 21.0])
t_u = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9,
                    33.9, 21.8, 48.4, 60.4, 68.4])
t_un = 0.1 * t_u

In [46]:
def model(t_u, w1, w2, b):
    return w1 * t_u ** 2 + w1 * t_u + b

In [47]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [48]:
import torch.optim as optim

dir(optim)

['ASGD',
 'Adadelta',
 'Adagrad',
 'Adam',
 'AdamW',
 'Adamax',
 'LBFGS',
 'NAdam',
 'Optimizer',
 'RAdam',
 'RMSprop',
 'Rprop',
 'SGD',
 'SparseAdam',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_functional',
 '_multi_tensor',
 'lr_scheduler',
 'swa_utils']

In [49]:
params1 = torch.tensor([1.0], requires_grad=True)
params2 = torch.tensor([1.0, 0.0], requires_grad=True)
learning_rate = 1e-5
optimizer = optim.SGD([params1, params2], lr=learning_rate)

In [50]:
t_p = model(t_u, *params1, *params2)
loss = loss_fn(t_p, t_c)
loss.backward()

optimizer.step()

params1
#params2

tensor([-234.1053], requires_grad=True)

In [51]:
params1 = torch.tensor([1.0], requires_grad=True)
params2 = torch.tensor([1.0, 0.0], requires_grad=True)
learning_rate = 1e-5
optimizer = optim.SGD([params1, params2], lr=learning_rate)

t_p = model(t_u, *params1, *params2)
loss = loss_fn(t_p, t_c)

optimizer.zero_grad() # <1>
loss.backward()
optimizer.step()

params1

tensor([-234.1053], requires_grad=True)

In [52]:
def training_loop(n_epochs, optimizer, params1, params2, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        t_p = model(t_u, *params1, *params2)
        loss = loss_fn(t_p, t_c)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % 500 == 0:
            print('Epoch %d, Loss %f' % (epoch, float(loss)))

    return params1, params2

In [53]:
params1 = torch.tensor([1.0], requires_grad=True)
params2 = torch.tensor([1.0, 0.0], requires_grad=True)
learning_rate = 1e-4
optimizer = optim.SGD([params1, params2], lr=learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    params1 = params1, # <1>
    params2 = params2,
    t_u = t_un,
    t_c = t_c)

Epoch 500, Loss 9.578312
Epoch 1000, Loss 9.274845
Epoch 1500, Loss 8.984344
Epoch 2000, Loss 8.706262
Epoch 2500, Loss 8.440063
Epoch 3000, Loss 8.185242
Epoch 3500, Loss 7.941311
Epoch 4000, Loss 7.707805
Epoch 4500, Loss 7.484279
Epoch 5000, Loss 7.270305


(tensor([0.3666], requires_grad=True),
 tensor([ 1.0000, -1.1579], requires_grad=True))

In [56]:
params1 = torch.tensor([1.0], requires_grad=True)
params2 = torch.tensor([1.0, 0.0], requires_grad=True)
learning_rate = 5e-2
optimizer = optim.Adam([params1, params2], lr=learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    params1 = params1, # <1>
    params2 = params2,
    t_u = t_un,
    t_c = t_c)

Epoch 500, Loss 2.764358
Epoch 1000, Loss 2.478057
Epoch 1500, Loss 2.477598
Epoch 2000, Loss 2.477598
Epoch 2500, Loss 2.477598
Epoch 3000, Loss 2.477597
Epoch 3500, Loss 2.477597
Epoch 4000, Loss 2.477597
Epoch 4500, Loss 2.477597
Epoch 5000, Loss 2.477597


(tensor([0.4725], requires_grad=True),
 tensor([ 1.0000, -5.8412], requires_grad=True))

In [57]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices  # <1>

(tensor([ 9,  3, 10,  5,  7,  8,  2,  1,  4]), tensor([6, 0]))

In [58]:
train_t_u = t_u[train_indices]
train_t_c = t_c[train_indices]

val_t_u = t_u[val_indices]
val_t_c = t_c[val_indices]

train_t_un = 0.1 * train_t_u
val_t_un = 0.1 * val_t_u

In [61]:
def training_loop(n_epochs, optimizer, params1, params2, train_t_u, val_t_u,
                  train_t_c, val_t_c):
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(train_t_u, *params1, *params2) # <1>
        train_loss = loss_fn(train_t_p, train_t_c)

        val_t_p = model(val_t_u, *params1, *params2) # <1>
        val_loss = loss_fn(val_t_p, val_t_c)

        optimizer.zero_grad()
        train_loss.backward() # <2>
        optimizer.step()

        if epoch <= 3 or epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                  f" Validation loss {val_loss.item():.4f}")

    return params1, params2


In [65]:
params1 = torch.tensor([1.0], requires_grad=True)
params2 = torch.tensor([1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.Adam([params1, params2], lr=learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    params1 = params1,
    params2 = params2,
    train_t_u = train_t_un, # <1>
    val_t_u = val_t_un, # <1>
    train_t_c = train_t_c,
    val_t_c = val_t_c)

Epoch 1, Training loss 782.4936, Validation loss 195.6477
Epoch 2, Training loss 758.4429, Validation loss 191.0499
Epoch 3, Training loss 734.7817, Validation loss 186.5092
Epoch 500, Training loss 6.6019, Validation loss 10.7328
Epoch 1000, Training loss 5.1972, Validation loss 7.5984
Epoch 1500, Training loss 3.9432, Validation loss 4.9270
Epoch 2000, Training loss 3.0983, Validation loss 3.2903
Epoch 2500, Training loss 2.6634, Validation loss 2.6172
Epoch 3000, Training loss 2.4985, Validation loss 2.5071
Epoch 3500, Training loss 2.4555, Validation loss 2.5780
Epoch 4000, Training loss 2.4485, Validation loss 2.6419
Epoch 4500, Training loss 2.4479, Validation loss 2.6674
Epoch 5000, Training loss 2.4478, Validation loss 2.6734


(tensor([0.4769], requires_grad=True),
 tensor([ 1.0000, -6.0633], requires_grad=True))

In [ ]:
def training_loop(n_epochs, optimizer, params, train_t_u, val_t_u,
                  train_t_c, val_t_c):
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(train_t_u, *params)
        train_loss = loss_fn(train_t_p, train_t_c)

        with torch.no_grad(): # <1>
            val_t_p = model(val_t_u, *params)
            val_loss = loss_fn(val_t_p, val_t_c)
            assert val_loss.requires_grad == False # <2>

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()


In [ ]:
def calc_forward(t_u, t_c, is_train):
    with torch.set_grad_enabled(is_train):
        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)
    return loss